In [1]:
using Pkg; Pkg.update()
#using Pkg; Pkg.resolve()
using SplitApplyCombine
using ParallelTemperingMonteCarlo
using DelimitedFiles
using StaticArrays
using Random, Plots

using BenchmarkTools

    Updating registry at `~/.julia/registries/General`


    Updating git-repo `https://github.com/JuliaRegistries/General.git`


   Installed ADTypes ────────────── v0.2.1
   Installed CUDA_Runtime_jll ───── v0.9.1+0


   Installed ProgressMeter ──────── v1.8.0
   Installed CUDA_Driver_jll ────── v0.6.0+3
   Installed Unitful ────────────── v1.17.0


   Installed MutableArithmetics ─── v1.3.2
   Installed SciMLBase ──────────── v1.96.2
   Installed PkgVersion ─────────── v0.3.3


...

  30 dependencies successfully precompiled in 262 seconds. 108 already precompiled.
┌ Info: We haven't cleaned this depot up for a bit, running Pkg.gc()...
└ @ Pkg /cache/build/default-amdci4-3/julialang/julia-release-1-dot-8/usr/share/julia/stdlib/v1.8/Pkg/src/Pkg.jl:719
      Active

 manifest files: 3 found
      Active

 artifact files: 141 found
      Active scratchspaces: 6 found


     Deleted 45 package installations (21.534 MiB)
     Deleted 1 repo (200.641 KiB)


┌ Warning: backend `GR` is not installed.
└ @ Plots ~/.julia/packages/Plots/3BCH5/src/backends.jl:43


# Main Goal

Broadly speaking, each potential requires different variables, vectors and changes at each step, this leads to method double-ups (with nonspecific variable types) as well as the need to rewrite high level functions such as mc_step and mc_cycle. By writing a single vector or struct that contains all the required vectors etc would solve both problems. 

In [2]:
abstract type PotentialVariables end

Each potential requires different vectors, both for sampling and as temporary delta vectors. Starting with the ELJ potential, requiring essentially nothing, maybe the new_dist2_vector if anything


In [3]:
mutable struct DimerPotentialVariables <: PotentialVariables
    new_dist2_vec::Vector
end
function DimerPotentialVariables(n_atoms::Int)
    new_dist2_vector = zeros(N_atoms)
    return DimerPotentialVariable(new_dist2_vector)
end

The NNPState struct becomes redundant, since we can add the various vectors to the potential variables

In [ ]:
mutable struct NNPVariables <: PotentialVariables
    new_dist2_vec::Vector
    new_en_atom::Vector
    g_matrix::Array
    f_matrix::Array
    new_g_matrix::Array
    new_f_vec::Vector
end
function NNPVariables(config,dist2_mat,n_atoms,pot::RuNNerPotential)
    f_matrix = cutoff_function.(sqrt.(dist2_mat),Ref(pot.r_cut))
    g_matrix = total_symm_calc(config.pos,dist2_mat,f_matrix,pot.symmetryfunctions)
    
    return NNPVariables(zeros(n_atoms),zeros(n_atoms),g_matrix,f_matrix,zeros(length(pot.symmetryfunctions)), zeros(n_atoms))
end


Similarly, the embedded atom model is defined as

In [ ]:
mutable struct EmbeddedAtomVariables <: PotentialVariables
    new_dist2_vec::Vector
    component_vector
    new_component_vector
end
function EmbeddedAtomVariables(dist2_matrix,pot)
    
    return EmbeddedAtomVariables(zeros(size(dist2_matrix)[1]),component_vector,new_component_vector)
end

And the Magnetic Field potential

In [ ]:
mutable struct ELJPotentialB <: PotentialVariables
    new_dist2_vec::Vector
    tanmat::Vector
    new_tan_vec::Vector
end